In [1]:
%matplotlib inline

import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [74]:
# Load raw Seattle data
with open('../data/raw_seattle.csv', 'r') as csv_file:
    df = pd.read_csv(csv_file)
    
del df['Notes']  # A useless column

In [75]:
df.columns

Index(['Jobtitle', 'Female Avg Hrly Rate', 'No. Female Empl',
       'Average of Female MONTHS LONGEVITY IN CURRENT CLASSIFICATION ',
       'Male Avg Hrly Rate', 'No. Male Empl',
       'Average of Male MONTHS LONGEVITY IN CURRENT CLASSIFICATION ',
       'Total Avg Hrly Rate', 'Total No. Empl',
       'Total Average of MONTHS LONGEVITY IN CURRENT CLASSIFICATION ',
       'Ratio of women's hourly rate to men's hourly rate - percentage'],
      dtype='object')

In [76]:
# Shorten the names of the columns
df = df.rename(columns={'Female Avg Hrly Rate': 'F Rate',
                        'No. Female Empl': '#F Empl',
                        'Average of Female MONTHS LONGEVITY IN CURRENT CLASSIFICATION ': 'Avg. F Long.',
                        'Male Avg Hrly Rate': 'M Rate',
                        'No. Male Empl': '#M Empl',
                        'Average of Male MONTHS LONGEVITY IN CURRENT CLASSIFICATION ': 'Avg. M Long.',
                        'Total Avg Hrly Rate': 'Total Rate',
                        'Total No. Empl': 'Total # Empl',
                        'Total Average of MONTHS LONGEVITY IN CURRENT CLASSIFICATION ': 'Total Avg. Long.',
                        "Ratio of women's hourly rate to men's hourly rate - percentage": '(W Rate / M Rate)'})

# Convert the percent signs in the last column to numbers
def convert_pct(x):
    if isinstance(x, str) and x[-1] == '%':
        return float(x[:-1]) / 100.0
    else:
        return x

df['(W Rate / M Rate)'] = df['(W Rate / M Rate)'].apply(convert_pct)

# Convert the other numeric columns to numbers
numeric_cols = ['F Rate',
                '#F Empl',
                'Avg. F Long.',
                'M Rate',
                '#M Empl',
                'Avg. M Long.',
                'Total Rate',
                'Total # Empl',
                'Total Avg. Long.']

df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric)

In [79]:
df = df.dropna()  # Remove missing values
df = df[df['Total # Empl'] > 1]  # Departments with only one employee don't give a good representation
df = df[:-1]  # Remove grand total line since it would throw off regression

In [77]:
# df.to_csv('../data/seattle_partially_cleaned.csv')

In [84]:
df_without_redundancy = df.copy()
del df_without_redundancy['Total Rate']
del df_without_redundancy['Total # Empl']
del df_without_redundancy['Total Avg. Long.']
df_without_redundancy

,Jobtitle,F Rate,#F Empl,Avg. F Long.,M Rate,#M Empl,Avg. M Long.,(W Rate / M Rate)
0,Accountant,30.58,23.0,88.96,30.28,7.0,85.71,1.0100
1,"Accountant,Prin",38.75,23.0,59.96,37.64,7.0,7.00,1.0295
2,"Accountant,Sr",34.48,23.0,60.61,33.89,6.0,43.67,1.0175
3,Act Exec,43.10,4.0,146.00,42.02,7.0,106.29,1.0256
5,Actg Tech I-BU,22.56,10.0,81.80,22.56,2.0,35.50,1.0000
6,Actg Tech II,23.98,7.0,112.71,24.35,1.0,75.00,0.9849
7,Actg Tech II-BU,24.19,74.0,103.23,24.21,18.0,134.50,0.9995
9,Actg Tech III-BU,26.69,32.0,89.97,26.18,5.0,51.20,1.0196
12,Admin Spec I,22.35,15.0,129.40,21.76,2.0,46.00,1.0270
13,Admin Spec I-BU,22.43,78.0,121.04,22.43,12.0,91.17,1.0001
